In [33]:
pip install psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [35]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
from IPython.display import HTML, display
from dotenv import load_dotenv
import os

In [36]:
load_dotenv()
DB_USER= os.getenv("DB_USER")
DB_PASS = os.getenv("DB_PASS")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

SYS_NAMES = ["prelude", "prelude-upgrade6", "clone"] 
connection_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)

In [37]:
sql_query = "SELECT * FROM ELN_WRITEUP_SCRAPPED;"

df = pd.read_sql(sql_query, engine)
df.head()

,exp_id,created_date,system_name,reactants_table,solvents_table,products_table,write_up
0,193403,2022-01-13,prelude,"<table id=""16044"" class=""tablescroll tableDefa...","<table id=""882893"" class=""tablescroll tableDef...","<table id=""16130"" class=""tablescroll tableDefa...","<p style=""font-size:11pt;"">To a solution of&nb..."
1,189001,2021-10-29,prelude,"<table id=""16044"" class=""tablescroll tableDefa...","<table id=""882893"" class=""tablescroll tableDef...","<table id=""16130"" class=""tablescroll tableDefa...","<p style=""font-size:11pt;color: #000000; font-..."
2,147003,2018-07-02,prelude,"<table id=""16044"" class=""tablescroll tableDefa...","<table id=""882893"" class=""tablescroll tableDef...","<table id=""16130"" class=""tablescroll tableDefa...","<p style=""font-size:11pt;line-height: 150%;"">&..."
3,138774,2016-10-03,prelude,"<table id=""16044"" class=""tablescroll tableDefa...","<table id=""882893"" class=""tablescroll tableDef...","<table id=""16130"" class=""tablescroll tableDefa...","<p style=""font-size:11pt;"">A mixture of[(3aR,4..."
4,192436,2021-12-21,prelude-upgrade6,"<table id=""16044"" class=""table_scroll_cont tab...","<table id=""882893"" class=""table_scroll_cont ta...","<table id=""16130"" class=""table_scroll_cont tab...","<p style=""font-size:11pt;""><span style=""font-s..."


In [17]:
def remove_styles(html_output):
    soup = BeautifulSoup(html_output, 'html.parser')
    
    for tag in soup.find_all(style=True):
        del tag['style']
    return soup.prettify()

In [28]:
central_css = """
<style>
/* Container for each experiment */
.experiment-container {
    border: 1px solid #ccc;
    margin: 10px 0;
    padding: 10px;
    background-color: #f9f9f9;
}

/* Header section for experiment ID and date */
.experiment-header {
    margin-bottom: 10px;
}

/* Flexbox container for columns */
.column-container {
    display: flex;
    flex-wrap: nowrap;
    gap: 10px;
}

/* Common styles for each column */
.column {
    flex: 1;
    border: 1px solid #ddd;
    padding: 10px;
}

/* Horizontal scrolling for the tables column */
.tables-column {
    overflow-x: auto;
    white-space: nowrap;
}

/* Vertical scrolling for write-up columns */
.writeup-column {
    overflow-y: auto;
    max-height: 300px; /* Optional: Limit height for scrolling */
}

/* Hidden content toggle visibility */
.hidden {
    display: none;
}

.visible {
    display: block;
}

/* Spacing for toggled content */
.toggled-content {
    margin-top: 10px;
}

</style>
"""

In [31]:
def render_html_group(idx, group):
    exp_id = group['exp_id'].iloc[0]
    created_date = group['created_date'].iloc[0]

    write_up_prod = remove_styles(group.loc[group['system_name'] == SYS_NAMES[0], 'write_up'].values[0] if SYS_NAMES[0] in group['system_name'].values else '<div>non-existent write-up</div>')
    write_up_up6 = remove_styles(group.loc[group['system_name'] == SYS_NAMES[1], 'write_up'].values[0] if SYS_NAMES[1] in group['system_name'].values else '<div>non-existent write-up</div>')
    # write_up_clone = group.loc[group['system_name'] == SYS_NAMES[2], 'write_up'].values[0] if SYS_NAMES[2] in group['system_name'].values else '<div>non-existent write-up</div>'

    return f"""
    {central_css}
    <div class="experiment-container">
    <div class="experiment-header">
        <strong>Row: </strong> {idx} <br>
        <strong>Experiment ID:</strong> {exp_id} <br>
        <strong>Created Date:</strong> {created_date}
    </div>
    <div class="column-container">
        <div class="column tables-column">
            <h3>Chemical Reagents Table</h3>
            <button onclick="toggleVisibility('reactants-{exp_id}')">Reactants</button>
            <div id="reactants-{exp_id}" class="hidden toggled-content">{remove_styles(group['reactants_table'].iloc[0])}</div>

            <button onclick="toggleVisibility('solvents-{exp_id}')">Solvents</button>
            <div id="solvents-{exp_id}" class="hidden toggled-content">{remove_styles(group['solvents_table'].iloc[0])}</div>

            <button onclick="toggleVisibility('products-{exp_id}')">Products</button>
            <div id="products-{exp_id}" class="hidden toggled-content">{remove_styles(group['products_table'].iloc[0])}</div>
        </div>
        <div class="column writeup-column">
            <h3>Production Write-Up</h3>
            {write_up_prod}
        </div>
        <div class="column writeup-column">
            <h3>Upgrade6 Write-Up</h3>
            {write_up_up6}
        </div>
        </div>
    </div>
"""

idx = 0
html_output = ""
for exp_id, group in df.groupby('exp_id'):
    idx += 1
    html_output += render_html_group(idx, group)
html_output += """
<script>
function toggleVisibility(id) {
    const element = document.getElementById(id);
    if (element.style.display === "none") {
        element.style.display = "block";
    } else {
        element.style.display = "none";
    }
}
</script>
"""
display(HTML(html_output))